In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib

from sklearn.tree import export_graphviz
import graphviz

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from statsmodels.api import Logit

from sklearn.tree import export_graphviz
import graphviz

from sklearn.tree import DecisionTreeClassifier

# 
# 1. 데이터 전처리

- null값 제거
- (.) 값 제거
- process 데이터와 tat 데이터 합치기
- tray 별 20미만의 셀 보유한 tray 데이터 삭제

In [2]:
#데이터 불러오기

data = pd.read_csv('bat_process.csv',encoding='cp949')

In [3]:
data.head()

,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,ocv2_deltaocv,ocv1_deltaocv,sa4_box_row,sa4_box_col,sa4_box_dan,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an
0,LOT-100,TRAY-1023,09-01-2020 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,...,5.0,6.8,07열,27연,03단,4718.0,37583.0,94.0,565.0,164.0
1,LOT-100,TRAY-1023,09-01-2020 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,...,5.0,6.8,07열,27연,03단,4718.0,37574.0,94.0,565.0,73.0
2,LOT-100,TRAY-1023,09-01-2020 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,...,5.1,6.9,07열,27연,03단,4700.0,37579.0,91.0,585.0,-57.0
3,LOT-100,TRAY-1023,09-01-2020 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,...,5.1,6.9,07열,27연,03단,4688.0,37583.0,93.0,606.0,99.0
4,LOT-100,TRAY-1023,09-01-2020 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,...,5.0,6.8,07열,27연,03단,4696.0,37570.0,91.0,605.0,36.0


In [4]:
#데이터가 커서, 전체를 다 확인하고자, 전체 확인 가능하도록 코드

pd.set_option("display.max_rows", None, "display.max_columns", None)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44217 entries, 0 to 44216
Columns: 112 entries, lot_id to m1_voltage_an
dtypes: float64(10), int64(54), object(48)
memory usage: 37.8+ MB


In [6]:
#결측값 확인

data.isnull().sum()

lot_id                0
tray_id               0
dt_start              0
cell_id               0
judge                 0
rta1_cell_no          0
rta1_box_col          0
rta1_box_row          0
rta1_box_dan          0
hta1_box_col          0
hta1_box_row          0
hta1_box_dan          0
rta2_box_col          0
rta2_box_row          0
rta2_box_dan          0
ocv1_ocv              0
ocv1_box_col          0
ocv1_box_dan          0
c1_curr_end           0
c1_voltage_avg        0
c1_capa               0
c1_ccval              0
c1_time_cc            0
c1_box_col            0
c1_box_dan            0
c1_temp_avg           0
dc1_curr_end          0
dc1_voltage_avg       0
dc1_capa              0
dc1_box_col           0
dc1_box_dan           0
dc1_temp_avg          0
dc1_capafit           0
c2_curr_end           0
c2_voltage_avg        0
c2_capa               0
c2_ccval              0
c2_time_cc            0
c2_box_col            0
c2_box_dan            0
c2_temp_avg           0
dc2_curr_end    

In [7]:
#결측값 처리

data.dropna(axis=0, inplace=True)

In [8]:
#결측값 확인

data.isnull().sum()

lot_id               0
tray_id              0
dt_start             0
cell_id              0
judge                0
rta1_cell_no         0
rta1_box_col         0
rta1_box_row         0
rta1_box_dan         0
hta1_box_col         0
hta1_box_row         0
hta1_box_dan         0
rta2_box_col         0
rta2_box_row         0
rta2_box_dan         0
ocv1_ocv             0
ocv1_box_col         0
ocv1_box_dan         0
c1_curr_end          0
c1_voltage_avg       0
c1_capa              0
c1_ccval             0
c1_time_cc           0
c1_box_col           0
c1_box_dan           0
c1_temp_avg          0
dc1_curr_end         0
dc1_voltage_avg      0
dc1_capa             0
dc1_box_col          0
dc1_box_dan          0
dc1_temp_avg         0
dc1_capafit          0
c2_curr_end          0
c2_voltage_avg       0
c2_capa              0
c2_ccval             0
c2_time_cc           0
c2_box_col           0
c2_box_dan           0
c2_temp_avg          0
dc2_curr_end         0
dc2_voltage_avg      0
dc2_capa   

In [12]:
#tray당 20 미만의 셀을 갖는 tray 확인

k=0
for i in range(1002,2792) :
    c = data[data['tray_id']=='TRAY-'+str(i)]['rta1_cell_no'].count()
    if c < 20 :
        k += c
        print('TRAY'+str(i), c)
        
print(k, k/len(data)) #(전체 데이터 중 111개의 행을 삭제, +0.002 정도의 적은 비율이므로 삭제로 판단)

TRAY1341 17
TRAY1342 8
TRAY1429 14
TRAY1439 0
TRAY1518 4
TRAY1652 18
TRAY1675 18
TRAY1841 19
TRAY2021 13
111 0.002515865820489574


In [14]:
#데이터에서 tray당 20 미만의 셀을 갖는 값들을 제거
#(결측값을 제거하기 전에는 22,23인 값들이 20이 되므로, 처음 작업 상황에선 큰 차이가 없다고 생각하므로, 20미만까지로만 제거)



for i in range(1002,2792) :
    c = data[data['tray_id']=='TRAY-'+str(i)]['rta1_cell_no'].count()
    if c < 20 :
        drop_=list(data[data['tray_id']=='TRAY-'+str(i)].index)
        data.drop(drop_, axis=0, inplace=True)
    drop_=[]


In [15]:
#시간에 따른 데이터 불러오기

tat = pd.read_csv('bat_tat.csv',encoding='cp949')

In [16]:
#결측값 확인

tat.isnull().sum()

lot_id        0
tray_id       0
dt_start      0
cell_id       0
rta1_tat      0
hta1_tat      0
rta2_tat      0
ocv1_tat      0
c1_tat        0
dc1_tat       0
c2_tat        0
dc2_tat       0
c3_tat        0
dc3_tat       0
c4_tat        0
ocv2_tat      0
pg1_tat       0
pc1_tat       0
sa1_tat       0
socv1_tat     0
sa2_tat      27
socv2_tat    27
sa3_tat      27
socv3_tat    27
sa4_tat      27
dtype: int64

In [17]:
#결측값 제거

tat.dropna(axis=0, inplace=True)

In [18]:
#결측값 제거됨 확인

tat.isnull().sum()

lot_id       0
tray_id      0
dt_start     0
cell_id      0
rta1_tat     0
hta1_tat     0
rta2_tat     0
ocv1_tat     0
c1_tat       0
dc1_tat      0
c2_tat       0
dc2_tat      0
c3_tat       0
dc3_tat      0
c4_tat       0
ocv2_tat     0
pg1_tat      0
pc1_tat      0
sa1_tat      0
socv1_tat    0
sa2_tat      0
socv2_tat    0
sa3_tat      0
socv3_tat    0
sa4_tat      0
dtype: int64

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44009 entries, 0 to 44216
Columns: 112 entries, lot_id to m1_voltage_an
dtypes: float64(10), int64(54), object(48)
memory usage: 39.2+ MB


In [20]:
tat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44190 entries, 0 to 44216
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   lot_id     44190 non-null  object 
 1   tray_id    44190 non-null  object 
 2   dt_start   44190 non-null  object 
 3   cell_id    44190 non-null  object 
 4   rta1_tat   44190 non-null  int64  
 5   hta1_tat   44190 non-null  int64  
 6   rta2_tat   44190 non-null  int64  
 7   ocv1_tat   44190 non-null  int64  
 8   c1_tat     44190 non-null  int64  
 9   dc1_tat    44190 non-null  int64  
 10  c2_tat     44190 non-null  int64  
 11  dc2_tat    44190 non-null  int64  
 12  c3_tat     44190 non-null  int64  
 13  dc3_tat    44190 non-null  int64  
 14  c4_tat     44190 non-null  int64  
 15  ocv2_tat   44190 non-null  int64  
 16  pg1_tat    44190 non-null  int64  
 17  pc1_tat    44190 non-null  int64  
 18  sa1_tat    44190 non-null  int64  
 19  socv1_tat  44190 non-null  int64  
 20  sa2_ta

In [22]:
#데이터 두 개 합침
#(.)에 대한 제거가 data에선 97행, tat에선 27개이므로, 70개의 null값과 tray 제거로 인한(null제거 이후이므로 +) 111개의 null값이 생김

concat_ = pd.concat([data, tat], axis=1)
concat_.isnull().sum()

#따라서, 총 181개의 결측값 존재

lot_id               181
tray_id              181
dt_start             181
cell_id              181
judge                181
rta1_cell_no         181
rta1_box_col         181
rta1_box_row         181
rta1_box_dan         181
hta1_box_col         181
hta1_box_row         181
hta1_box_dan         181
rta2_box_col         181
rta2_box_row         181
rta2_box_dan         181
ocv1_ocv             181
ocv1_box_col         181
ocv1_box_dan         181
c1_curr_end          181
c1_voltage_avg       181
c1_capa              181
c1_ccval             181
c1_time_cc           181
c1_box_col           181
c1_box_dan           181
c1_temp_avg          181
dc1_curr_end         181
dc1_voltage_avg      181
dc1_capa             181
dc1_box_col          181
dc1_box_dan          181
dc1_temp_avg         181
dc1_capafit          181
c2_curr_end          181
c2_voltage_avg       181
c2_capa              181
c2_ccval             181
c2_time_cc           181
c2_box_col           181
c2_box_dan           181


In [23]:
#결측값 처리

concat_.dropna(axis=0, inplace=True)

In [24]:
#결측값 처리 확인

concat_.isnull().sum()

lot_id               0
tray_id              0
dt_start             0
cell_id              0
judge                0
rta1_cell_no         0
rta1_box_col         0
rta1_box_row         0
rta1_box_dan         0
hta1_box_col         0
hta1_box_row         0
hta1_box_dan         0
rta2_box_col         0
rta2_box_row         0
rta2_box_dan         0
ocv1_ocv             0
ocv1_box_col         0
ocv1_box_dan         0
c1_curr_end          0
c1_voltage_avg       0
c1_capa              0
c1_ccval             0
c1_time_cc           0
c1_box_col           0
c1_box_dan           0
c1_temp_avg          0
dc1_curr_end         0
dc1_voltage_avg      0
dc1_capa             0
dc1_box_col          0
dc1_box_dan          0
dc1_temp_avg         0
dc1_capafit          0
c2_curr_end          0
c2_voltage_avg       0
c2_capa              0
c2_ccval             0
c2_time_cc           0
c2_box_col           0
c2_box_dan           0
c2_temp_avg          0
dc2_curr_end         0
dc2_voltage_avg      0
dc2_capa   

In [25]:
concat_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44009 entries, 0 to 44216
Columns: 137 entries, lot_id to sa4_tat
dtypes: float64(69), int64(16), object(52)
memory usage: 46.3+ MB


In [26]:
concat_.head()

,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,hta1_box_row,hta1_box_dan,rta2_box_col,rta2_box_row,rta2_box_dan,ocv1_ocv,ocv1_box_col,ocv1_box_dan,c1_curr_end,c1_voltage_avg,c1_capa,c1_ccval,c1_time_cc,c1_box_col,c1_box_dan,c1_temp_avg,dc1_curr_end,dc1_voltage_avg,dc1_capa,dc1_box_col,dc1_box_dan,dc1_temp_avg,dc1_capafit,c2_curr_end,c2_voltage_avg,c2_capa,c2_ccval,c2_time_cc,c2_box_col,c2_box_dan,c2_temp_avg,dc2_curr_end,dc2_voltage_avg,dc2_capa,dc2_box_col,dc2_box_dan,dc2_temp_avg,dc2_capafit,c3_curr_end,c3_voltage_avg,c3_capa,c3_ccval,c3_time_cv,c3_cvval,c3_time_cc,c3_box_col,c3_box_dan,c3_temp_avg,dc3_curr_end,dc3_voltage_avg,dc3_capa,dc3_box_col,dc3_box_dan,dc3_temp_avg,dc3_capafit,c4_curr_end,c4_voltage_avg,c4_capa,c4_ccval,c4_time_cv,c4_cvval,c4_time_cc,c4_box_col,c4_box_dan,c4_temp_avg,ocv2_ocv,ocv2_box_col,ocv2_box_dan,pg1_impfit,pg1_imp,pg1_voltage_start,pg1_curr_end,pg1_box_col,pg1_box_dan,pg1_temp_avg,pc1_imp,pc1_voltage_start,pc1_curr_end,pc1_box_col,pc1_box_dan,sa1_box_col,sa1_box_row,sa1_box_dan,socv1_ocv,sa2_box_col,sa2_box_row,sa2_box_dan,socv2_ocv,sa3_box_col,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,sa4_box_row,sa4_box_col,sa4_box_dan,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an,lot_id,tray_id,dt_start,cell_id,rta1_tat,hta1_tat,rta2_tat,ocv1_tat,c1_tat,dc1_tat,c2_tat,dc2_tat,c3_tat,dc3_tat,c4_tat,ocv2_tat,pg1_tat,pc1_tat,sa1_tat,socv1_tat,sa2_tat,socv2_tat,sa3_tat,socv3_tat,sa4_tat
0,LOT-100,TRAY-1023,09-01-2020 오전 10:01:25,CELL-10747,양품,10.0,08열,33연,08단,03열,25연,03단,04열,64연,06단,3333.0,22열,08단,6801.0,3781.0,6451.0,6451.0,3417.0,22열,08단,268.0,6799.0,4108.0,544.0,22열,08단,266.0,539.0,6800.0,4146.0,625.0,625.0,332.0,22열,08단,265.0,6799.0,4112.0,544.0,22열,08단,266.0,539.0,135.0,4189.0,749.0,578.0,1190.0,171.0,307.0,22열,08단,266.0,6796.0,3674.0,7094.0,22열,08단,269.0,7088.0,136.0,3629.0,4282.0,4072.0,1091.0,210.0,2157.0,22열,08단,273.0,3764.0,22열,08단,2211.0,2114.0,3759.0,120.0,12열,01단,288.0,2086.0,3714.0,150.0,12열,01단,09열,42연,01단,3766.1,03열,64연,05단,3761.1,14열,37연,04단,3759.3,5.0,6.8,07열,27연,03단,4718.0,37583.0,94.0,565.0,164.0,LOT-100,TRAY-1023,09-01-2020 오전 10:01:25,CELL-10747,217066,86461,5732,119,3556,404,466,404,1797,3884,3428,118,290,290,53500,5,355357.0,5.0,215150.0,5.0,207532.0
1,LOT-100,TRAY-1023,09-01-2020 오전 10:01:25,CELL-10748,양품,7.0,08열,33연,08단,03열,25연,03단,04열,64연,06단,3329.0,22열,08단,6797.0,3783.0,6390.0,6390.0,3385.0,22열,08단,268.0,6800.0,4103.0,545.0,22열,08단,266.0,540.0,6802.0,4144.0,657.0,657.0,349.0,22열,08단,265.0,6800.0,4110.0,544.0,22열,08단,266.0,539.0,136.0,4189.0,768.0,588.0,1286.0,180.0,313.0,22열,08단,266.0,6799.0,3674.0,7091.0,22열,08단,269.0,7085.0,136.0,3624.0,4268.0,4064.0,969.0,204.0,2153.0,22열,08단,273.0,3763.0,22열,08단,2224.0,2127.0,3758.0,120.0,12열,01단,288.0,2093.0,3714.0,150.0,12열,01단,09열,42연,01단,3765.2,03열,64연,05단,3760.2,14열,37연,04단,3758.4,5.0,6.8,07열,27연,03단,4718.0,37574.0,94.0,565.0,73.0,LOT-100,TRAY-1023,09-01-2020 오전 10:01:25,CELL-10748,217066,86461,5732,119,3556,404,466,404,1797,3884,3428,118,290,290,53500,5,355357.0,5.0,215150.0,5.0,207532.0
2,LOT-100,TRAY-1023,09-01-2020 오전 10:01:25,CELL-10749,양품,19.0,08열,33연,08단,03열,25연,03단,04열,64연,06단,3329.0,22열,08단,6798.0,3781.0,6436.0,6436.0,3410.0,22열,08단,268.0,6799.0,4107.0,544.0,22열,08단,266.0,539.0,6801.0,4146.0,630.0,630.0,335.0,22열,08단,265.0,6799.0,4111.0,544.0,22열,08단,266.0,539.0,136.0,4189.0,753.0,579.0,1228.0,174.0,308.0,22열,08단,266.0,6799.0,3674.0,7079.0,22열,08단,269.0,7073.0,136.0,3630.0,4270.0,4058.0,1112.0,212.0,2150.0,22열,08단,273.0,3764.0,22열,08단,2217.0,2120.0,3759.0,120.0,12열,01단,288.0,2081.0,3715.0,150.0,12열,01단,09열,42연,01단,3765.9,03열,64연,05단,3760.8,14열,37연,04단,3759.0,5.1,6.9,07열,27연,03단,4700.0,37579.0,91.0,585.0,-57.0,LOT-100,TRAY-1023,09-01-2020 오전 10:01:25,CELL-10749,217066,86461,5732,118,3556,405,466,404,1797,3884,3428,118,290,290,53500,5,355357.0,5.0,215150.0,5.0,207532.0
3,LOT-100,TRAY-1023,09-01-2020 오전 10:01:25,CELL-10750,양품,24.0,08열,33연,08단,03열,25연,03단,04열

In [134]:
#데이터 따로 저장

concat_.to_csv('인호오빠.csv')

# 
# 2. EDA 
- 열, 연, 단 (설비공정)에 의한 EDA
- 불량률 차이가 큰 설비 간의 변수에 따른 공정 변수간 유의차 확인
- 차이 있는 변수에 대한 불량률의 상관관계 파악
- 그에 따른 주요 변수 파악 및 해석

# 
# 3. Decision Tree
- DT 분류 모델을 이용한, 주요 변수 도출
- EDA로 도출한 변수와 함께 주요 변수 도출

# 
# 4. 예측 모델
- 앞서 구한 주요한 변수로 예측 모델 생성
- Decision Tree
- Random Forest
- XG boost
- 이후 ? Ada 모델 추가?

# 
# 5. 주성분 분석